#### imports etc

In [58]:
# imports
from lxml import html, etree
import requests
import pandas as pd
import datetime as dt

## Function

The purpose of the function is to extract data from the listings on the http://www.rightmove.co.uk/ property website. After passing the function the long url from the first results page of a search, the function will extract the price, property type, address details, and url for the specific property listing. It will also extract the postcode stems from the address details (e.g. 'N1') and store this in a separate column; and extract the number of bedrooms from the property type as a separate column. If more than one page of results are returned by the search then the function will automatically cycle through all pages collecting the data (which means it can take a while to return the results if the search criteria returns thousands of results).

In [65]:
def rightmove_webscrape(rightmove_url,rent_or_buy):
    
# Get the start & end of the web url around the index value
    start,end = rightmove_url.split('&index=')
    url_start = start+'&index='
    url_end = end[1:]
    
# Initialise variables
    price_pcm=[]
    titles=[]
    addresses=[]
    weblinks=[]
    page_counts=[]
    
# Initialise pandas DataFrame for results.
    df=pd.DataFrame(columns=['price','type','address','url'])

# Get the total number of results from the search
    page = requests.get(rightmove_url)
    tree = html.fromstring(page.content)
    xp_result_count = '//span[@class="searchHeader-resultCount"]/text()'
    result_count = int(tree.xpath(xp_result_count)[0].replace(",", ""))
    
# Turn the total number of search results into number of iterations for the loop
    loop_count = result_count/24
    if result_count%24>0:
        loop_count = loop_count+1
        
# Set the Xpath variables for the loop
    if rent_or_buy=='rent':
        xp_prices = '//span[@class="propertyCard-priceValue"]/text()'
    elif rent_or_buy=='buy':
        xp_prices = '//div[@class="propertyCard-priceValue"]/text()'
        
    xp_titles = '//div[@class="propertyCard-details"]//a[@class="propertyCard-link"]//h2[@class="propertyCard-title"]/text()'
    xp_addresses = '//address[@class="propertyCard-address"]/text()'
    xp_weblinks = '//div[@class="propertyCard-details"]//a[@class="propertyCard-link"]/@href'

# Start the loop through the search result web pages
    for pages in range(0,loop_count,1):
        rightmove_url = url_start+str(pages*24)+url_end
        page = requests.get(rightmove_url)
        tree = html.fromstring(page.content)
        
# Reset variables
        price_pcm=[]
        titles=[]
        addresses=[]
        weblinks=[]

# Create data lists from Xpaths
        for val in tree.xpath(xp_prices):
            price_pcm.append(val)
        for val in tree.xpath(xp_titles):
            titles.append(val)
        for val in tree.xpath(xp_addresses):
            addresses.append(val)
        for val in tree.xpath(xp_weblinks):
            weblinks.append(val)

# Convert data to temporary DataFrame
        data = [price_pcm, titles, addresses, weblinks]
        temp_df= pd.DataFrame(data)
        temp_df = temp_df.transpose()
        temp_df.columns=['price','type','address','url']
        
# Drop empty rows from DataFrame which come from placeoholders in html file.
        temp_df = temp_df[temp_df.url != '/property-for-sale/property-0.html']
    
# Join temporary DataFrame to main results DataFrame.
        frames = [df,temp_df]
        df = pd.concat(frames)

# Renumber results DataFrame index to remove duplicate index values.
    df = df.reset_index(drop=True)

# Convert price column to numeric values for analysis.
    df.price.replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
    df.price=pd.to_numeric(df.price)

# Extract postcode areas to separate column.
    df['postcode'] = df['address'].str.extract(r'\b([A-Za-z][A-Za-z]?[0-9][0-9]?[A-Za-z]?)\b',expand=True)
    
# Extract number of bedrooms from 'type' column.
    df['number_bedrooms'] = df.type.str.extract(r'\b([\d][\d]?)\b',expand=True)
    df.loc[df['type'].str.contains('studio',case=False),'number_bedrooms']=0

# Add in search_date column with date website was queried (i.e. today's date).
    now = dt.datetime.today().strftime("%d/%m/%Y")
    df['search_date'] = now

# Optional line to export the results to CSV if you wish to inspect them in an alternative program.
#     df.to_csv('rightmove_df.csv',encoding='utf-8')

    print 'The search returned a total of ', len(df),' results.'
    return df

## Using the function

To use the function you first need to go to http://www.rightmove.co.uk/ and perform your search based on your desired criteria (e.g. 1 bedroom flats to rent in London Fields added to the website in the last 7 days). When the first page of results comes up copy the long url from the browser window and set it as the *url* variable (in this example the search is for all residential properties for sale in London fields).

The second argument to pass to the function is just a text value of 'buy' or 'rent' depending on what you searched for on rightmove. This argument ensures that the correct Xpaths are used in the function to produce the price values.

In [88]:
rent_url = 'http://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%5E70417&numberOfPropertiesPerPage=24&radius=0.0&sortType=6&index=0&propertyTypes=detached%2Csemi-detached%2Cterraced%2Cflat%2Cbungalow&includeLetAgreed=false&viewType=LIST&areaSizeUnit=sqft&currencyCode=GBP'

In [89]:
buy_url = 'http://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E70417&numberOfPropertiesPerPage=24&radius=0.0&sortType=2&index=0&includeSSTC=false&viewType=LIST&areaSizeUnit=sqft&currencyCode=GBP'

Simply run the function on the url variable to create the dataframe. Here we'll assign the results to the *df* variable:

In [90]:
url = buy_url
df = rightmove_webscrape(url,'buy')

The search returned a total of  154  results.


We can look at the first few rows of data to check that the function worked as expected:

In [91]:
df.head()

,price,type,address,url,postcode,number_bedrooms,search_date
0,550000.0,2 bedroom flat for sale,"Pritchards Road, London",/property-for-sale/property-61788488.html,NaN,2,16/09/2016
1,2350000.0,4 bedroom terraced house for sale,"Albion Square, Hackney, E8",/property-for-sale/property-61569422.html,E8,4,16/09/2016
2,2100000.0,4 bedroom terraced house for sale,"Elrington Road, Hackney",/property-for-sale/property-56114773.html,NaN,4,16/09/2016
3,2000000.0,Detached house for sale,"76 Shrubland Road, London, Hackney, E8 4NH",/property-for-sale/property-61276148.html,E8,NaN,16/09/2016
4,2000000.0,4 bedroom semi-detached house for sale,"Greenwood Road, London Fields, Hackney, London...",/property-for-sale/property-43998885.html,E8,4,16/09/2016


And export the full results to *csv* for analysis:

In [92]:
df.to_csv('search_results.csv',encoding='utf-8',index=False)

## Optional html export

In the event that the search does not return results as expected it may be that the Xpaths have changed and need updating; alternatively you may wish to add in additional Xpaths to collect more data. The below will export the full html text file from whichever url you set as the variable *rightmove_url*.

In [93]:
page = requests.get(rightmove_url)
tree = html.fromstring(page.content)
html_text=etree.tostring(tree)
file = open("html.txt", "w")
file.write(html_text)
file.close()